# 1. SVM-метод, работа с SVM-методом, нахождение наиболее важных объекты выборки и svm-dataset

### Инструкция по выполнению
- Загрузите выборку из файла svm-data.csv. В нем записана двумерная выборка (целевая переменная указана в первом столбце, признаки — во втором и третьем)  
- Обучите классификатор с линейным ядром, параметром C = 100000 и random_state=241. Такое значение параметра нужно использовать, чтобы убедиться, что SVM работает с выборкой как с линейно разделимой. При более низких значениях параметра алгоритм будет настраиваться с учетом слагаемого в функционале, штрафующего за маленькие отступы, из-за чего результат может не совпасть с решением классической задачи SVM для линейно разделимой выборки.   
- Найдите номера объектов, которые являются опорными (нумерация с единицы). Они будут являться ответом на задание. Обратите внимание, что в качестве ответа нужно привести номера объектов в возрастающем порядке через запятую или пробел. Нумерация начинается с 1.

In [1]:
import numpy as np
import pandas as pd

In [2]:
columns=['y','x1','x2']

In [3]:
data=pd.read_csv(r'./datasets/svm-data.csv',sep=',', names=columns)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
y     10 non-null float64
x1    10 non-null float64
x2    10 non-null float64
dtypes: float64(3)
memory usage: 320.0 bytes


In [4]:
data

,y,x1,x2
0,0.0,0.70,0.29
1,1.0,0.23,0.55
2,0.0,0.72,0.42
3,0.0,0.98,0.68
4,0.0,0.48,0.39
5,1.0,0.34,0.73
6,0.0,0.44,0.06
7,1.0,0.40,0.74
8,0.0,0.18,0.18
9,1.0,0.53,0.53


In [5]:
X = data[['x1','x2']]
y = data.y

In [6]:
from sklearn.svm import SVC
clf = SVC(C=100000,kernel='linear',random_state=241)
clf.fit(X, y)  

SVC(C=100000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False)

In [7]:
print('Опорные объекты-вектора:\n',clf.support_vectors_)
print('Номера объектов-опорных векторов:', clf.support_+1)

Опорные объекты-вектора:
 [[0.98 0.68]
 [0.48 0.39]
 [0.53 0.53]]
Номера объектов-опорных векторов: [ 4  5 10]


# 2. Анализ текстов с пом. SVM - определение, к какой теме относится текст



In [8]:
from sklearn import datasets
newsgroups = datasets.fetch_20newsgroups(
                    subset='all', 
                    categories=['alt.atheism', 'sci.space']
             )

После выполнения этого кода массив с текстами будет находиться в поле newsgroups.data, номер класса — в поле newsgroups.target.

Итак, нужно векторизовать текстовые данные. Есть разные векторайзеры. 
Авторы рекомендуют sklearn.feature_extraction.text.TfidfVectorizer(вычисление TF-IDF).

In [9]:
newsgroups.target

array([0, 0, 1, ..., 1, 1, 0], dtype=int64)

In [10]:
X=newsgroups.data
y=newsgroups.target
y

array([0, 0, 1, ..., 1, 1, 0], dtype=int64)

  
Некоторая предобработка, удаление ненужных '\n', которые искажают информацию. Закомментировала, так как в задании об этом не говорится и ответ может не сойтись:

In [11]:
'''
newsgroups.data = [line.splitlines() for line in newsgroups.data]
X=[]
#k=0
for i in newsgroups.data:
    i=''.join(i)
    X.append(i)
    #k+=1
'''

"\nnewsgroups.data = [line.splitlines() for line in newsgroups.data]\nX=[]\n#k=0\nfor i in newsgroups.data:\n    i=''.join(i)\n    X.append(i)\n    #k+=1\n"

In [12]:
#X

Вычислите TF-IDF-признаки для всех текстов. Обратите внимание, что в этом задании мы предлагаем вам вычислить TF-IDF по всем данным. При таком подходе получается, что признаки на обучающем множестве используют информацию из тестовой выборки — но такая ситуация вполне законна, поскольку мы не используем значения целевой переменной из теста. На практике нередко встречаются ситуации, когда признаки объектов тестовой выборки известны на момент обучения, и поэтому можно ими пользоваться при обучении алгоритма.

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
vectorizer = TfidfVectorizer()
X_vec = vectorizer.fit_transform(X)

In [15]:
feature_mapping = vectorizer.get_feature_names()
feature_mapping[10000]
#Пример: 10000 признаку соответствует этот признак, а все потому, что очень много чисел...

'do'

In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [17]:
grid = {'C': np.power(10.0, np.arange(-5, 5))}
cv = KFold(n_splits=5, shuffle=True, random_state=241)
clf = SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)
gs.fit(X_vec, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

Подберите минимальный лучший параметр C из множества [10^-5, 10^-4, ... 10^4, 10^5] для SVM с линейным ядром (kernel='linear') при помощи кросс-валидации по 5 блокам. Укажите параметр random_state=241 и для SVM, и для KFold. В качестве меры качества используйте долю верных ответов (accuracy).
Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге.

## Ищем лучший вариант:

In [19]:
gs.grid_scores_
    # a.mean_validation_score — оценка качества по кросс-валидации
    # a.parameters — значения параметров

C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:762: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.55263, std: 0.02812, params: {'C': 1e-05},
 mean: 0.55263, std: 0.02812, params: {'C': 0.0001},
 mean: 0.55263, std: 0.02812, params: {'C': 0.001},
 mean: 0.55263, std: 0.02812, params: {'C': 0.01},
 mean: 0.95017, std: 0.00822, params: {'C': 0.1},
 mean: 0.99328, std: 0.00455, params: {'C': 1.0},
 mean: 0.99328, std: 0.00455, params: {'C': 10.0},
 mean: 0.99328, std: 0.00455, params: {'C': 100.0},
 mean: 0.99328, std: 0.00455, params: {'C': 1000.0},
 mean: 0.99328, std: 0.00455, params: {'C': 10000.0}]

### 'mean: 0.99328, std: 0.00455, params: {'C': 1.0}' - попробую взять макс. с мин.С, чтобы была максимальная регуляризация и, как следствие, меньшая переобучаемость.

In [43]:
clf = SVC(C=1.0,kernel='linear',random_state=241)
clf.fit(X_vec, y) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False)

Найдите 10 слов с наибольшим абсолютным значением веса (веса хранятся в поле coef_ у svm.SVC). Они являются ответом на это задание. Укажите эти слова через запятую или пробел, в нижнем регистре, в лексикографическом порядке.

In [45]:
results = gs.best_estimator_.coef_ 
row = results.getrow(0).toarray()[0].ravel()
top_ten_indicies = np.argsort(abs(row))[-10:]

top_ten_values = row[top_ten_indicies]

for a in top_ten_indicies:
    print(feature_mapping[a])

sci
keith
bible
religion
sky
moon
atheists
atheism
god
space


Итог: atheism atheists bible god keith moon religion sci sky space